In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# I have experienced stability issues with float32.
from jax.config import config
config.update("jax_enable_x64", True)
config.update("jax_platform_name", "cpu")

import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]=".8"

In [3]:
import time
import matplotlib.pyplot as plt
import numpy as np
import jax
import jax.numpy as jnp
from jax import jit, value_and_grad

import jaxley as jx
from jaxley.channels import HHChannel
from jaxley.synapses import GlutamateSynapse

### Setup

In [4]:
# Number of segments per branch.
nseg_per_branch = 8

# Stimulus.
i_delay = 3.0  # ms
i_amp = 0.05  # nA
i_dur = 2.0  # ms

# Duration and step size.
dt = 0.025  # ms
t_max = 50.0  # ms

In [5]:
time_vec = jnp.arange(0.0, t_max+dt, dt)

### Define compartment and branch

In [6]:
comp = jx.Compartment()
branch = jx.Branch([comp for _ in range(nseg_per_branch)])

### Define cell

Below, we define the morphology. Each element of `parents` corresponds to one branch. A `-1` entry means that this branch does not have a parent. Alternatively, you could also load cells from SWC with 

```parents, lengths, endpoint_rads, start_rad = jx.utils.read_swc(fname, max_branch_len=300.0)```.

You can plot the SWC file with `jx.utils.plot_swc(fname, max_branch_len=300.0)`.

In [7]:
num_branches = 15
parents = jnp.asarray([-1, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6])
assert len(parents) == num_branches

In [8]:
cell = jx.Cell([branch for _ in range(num_branches)], parents=parents)

Cells can be visualized as follows:

In [9]:
_ = jx.utils.plot_morph(cell, figsize=(4, 4), cols="k", highlight_branch_inds=[0, 1, 6])

### Define connectivity

In this case, two layers fully connected.

In [10]:
num_cells = 16

In [11]:
conn_builder = jx.ConnectivityBuilder([cell.total_nbranches for _ in range(num_cells)])

_ = np.random.seed(0)  # location of post synapse is random.
connectivities = [
    jx.Connectivity(
        GlutamateSynapse(),
        [
            *conn_builder.fc(np.arange(0, 5), np.arange(5, 15)),
            *conn_builder.fc(np.arange(5, 15), np.arange(15, 16)),
        ]
    ),
]

In [12]:
network = jx.Network([cell for _ in range(num_cells)], connectivities)

### Insert mechanisms

In [13]:
network.insert(HHChannel())

### Define stimuli and recordings

In [14]:
for cell_ind in range(num_cells):
    network.cell(cell_ind).branch(0).comp(0.0).record()

current = jx.step_current(i_delay, i_dur, i_amp, time_vec)
for stim_ind in range(5):
    network.cell(stim_ind).branch(0).comp(0.0).stimulate(current)

### Solve ODE

In [15]:
%%time
s = jx.integrate(network, delta_t=dt)

CPU times: user 7.69 s, sys: 91.9 ms, total: 7.78 s
Wall time: 7.85 s


### jit compilation

We can jit-compile the simulation in order to make it faster. The code below will be slow when run for the first time, but fast upon the second run.

In [16]:
def run_sim():
    return jx.integrate(network, delta_t=dt)

jitted_run_sim = jit(run_sim)

In [17]:
%%time
s = jitted_run_sim()

CPU times: user 6.41 s, sys: 62.3 ms, total: 6.47 s
Wall time: 6.53 s


### Visualize traces

In [18]:
fig, ax = plt.subplots(1, 1, figsize=(6.3, 3))
for i in range(num_cells):
    _ = ax.plot(time_vec, s[i][:-1], c="k")
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Voltage (mV)")
plt.show()